In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import ndcg_score, average_precision_score

## Loading

In [2]:
anime = pd.read_csv('/kaggle/input/anime-recommendations-database/anime.csv', delimiter=',')
rating = pd.read_csv('/kaggle/input/anime-recommendations-database/rating.csv', delimiter=',')


In [3]:
print(anime.shape)
print(rating.shape)

(12294, 7)
(7813737, 3)


In [4]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [5]:
rating.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [6]:
# Clean ratings: remove or impute -1 ratings (unknown ratings)
print("Original shape:", rating.shape)
rating_clean = rating[rating['rating'] != -1]  # remove -1s
print("After removing -1s:", rating_clean.shape)

Original shape: (7813737, 3)
After removing -1s: (6337241, 3)


In [13]:
# Remove duplicate (user_id, anime_id) combinations by keeping the mean rating
rating_clean = rating_clean.groupby(['user_id', 'anime_id']).agg({'rating': 'mean'}).reset_index()

In [14]:
# Create user-item matrix
user_item_matrix = rating_clean.pivot(index='user_id', columns='anime_id', values='rating')

In [15]:
# Split into train and test
train, test = train_test_split(rating_clean, test_size=0.2, random_state=42)
train_matrix = train.pivot(index='user_id', columns='anime_id', values='rating')

In [16]:
# Simple user-based CF (mean rating of similar users)
def predict_rating(user_id, anime_id):
    if user_id not in train_matrix.index or anime_id not in train_matrix.columns:
        return np.nan

    user_ratings = train_matrix.loc[user_id]
    similar_users = train_matrix[anime_id].dropna().index
    if len(similar_users) == 0:
        return np.nan

    return train_matrix.loc[similar_users, anime_id].mean()

In [23]:
# Evaluation function with ranked list simulation
def evaluate(test_df, k=10):
    grouped = test_df.groupby('user_id')
    ndcg_scores = []
    map_scores = []

    for user_id, group in tqdm(grouped):
        true_animes = group['anime_id'].tolist()
        true_ratings = group['rating'].tolist()

        scored_items = []
        for anime_id, true_rating in zip(true_animes, true_ratings):
            pred = predict_rating(user_id, anime_id)
            if np.isnan(pred):
                continue
            scored_items.append((anime_id, true_rating, pred))

        if len(scored_items) < 2:
            continue

        # Sort by predicted score
        scored_items.sort(key=lambda x: x[2], reverse=True)

        # Select top-k
        top_k = scored_items[:k]
        true_relevance = [x[1] for x in top_k]  # real ratings as relevance
        predicted_scores = [x[2] for x in top_k]

        # Convert to required format (2D arrays)
        ndcg = ndcg_score([true_relevance], [predicted_scores], k=k)
        ndcg_scores.append(ndcg)

        # For MAP, convert true_relevance to binary: 1 if rating >= 7 else 0
        binary_relevance = [1 if r >= 7 else 0 for r in true_relevance]
        try:
            ap = average_precision_score(binary_relevance, predicted_scores)
            map_scores.append(ap)
        except ValueError:
            continue

    return np.mean(map_scores), np.mean(ndcg_scores)

In [ ]:
# Evaluation
map_score, ndcg_score_val = evaluate(test)
print(f"MAP: {map_score:.4f}, NDCG: {ndcg_score_val:.4f}")

  1%|          | 430/62390 [00:23<1:01:49, 16.70it/s]/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
  3%|▎         | 1798/62390 [01:43<1:01:08, 16.52it/s]/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
  4%|▍         | 2413/62390 [02:14<43:11, 23.14it/s]  /usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
  5%|▍         | 3004/62390 [02:52<1:12:13, 13.71it/s]/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
  7%|▋         | 4491/62390 [04:26<1:09:11, 13.95it/s]/usr/local/lib/

MAP: 0.9608, NDCG: 0.9773
